In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
import time
import math
# 数据预处理相关函数
from Reader import Footscan_reader
from Reader import gait_data_reader
from Convlstm import ConvLSTM
import random
LEARING_RATE=0.001
BATCH_SIZE=512
INPUT_FEATURES=5
HIDDEN_FEATURES_LIST=[64,128]
KERNEL_SIZE_LIST=[(1,1),(2,2),(3,3),(4,4),(3,3)]
EPOCH_NUM=200
DEVICE=torch.device('cuda')
TIME_STEP=202


## 首先先读数据，并作配准

In [2]:
'''之前的是对每一个gaitdata，找对应的name和week的所有趟的数据，然后选出对应的那一趟，
这样就导致每一个时间点的十趟数据被检索了十次，过于浪费时间，可以修改一下:
修改成，我先对有gait数据的name和week的组合做一个统计，然后对这个组合所有趟的足压数据都进行检索，
如果检索到了，再根据检索到的每一趟，反过来看这一趟有没有gait数据，然后做匹配录入'''

def get_name_week_list(gait_data_list):
    name_week_list=[]
    name_list=[]
    for gaitdata in gait_data_list:
        name_week=(info_dic.get(gaitdata.name),str(gaitdata.week*12))
        if name_week[0] in name_list:
            pass
        else:
            name_list.append(name_week[0])
        if name_week in name_week_list:
            pass
        else:
            name_week_list.append(name_week)
    print('该部分数据中共有 {} 位患者，平均每位患者有{}次回访，共计{}次,平均每次回访测试了{}组数据，共计{}组'.format(len(name_list),len(name_week_list)/len(name_list),len(name_week_list),len(gait_data_list)/len(name_week_list),len(gait_data_list)))
    print('\n \n')
    return name_week_list


info_dic={}
info=pd.read_excel('G:\\qinyue\\data\\DJO\\info.xlsx')
for i in range(len(info)):
    info_dic[info.loc[i,'编号']]=info.loc[i,'姓名']


footscan_reader = Footscan_reader()
gaitdata_reader=gait_data_reader()
gait_data_list=gaitdata_reader.get_data(gait_data_category='kneemoment')
valid_input=[]
valid_gaitdata=[]
valid_week=[]
valid_delta=[]


for name_week in get_name_week_list(gait_data_list):
    #寻找这次回访的所有趟的足压数据
    #此时的week就是这组数据隶属的task——-12，-24，-36，-48，12，24，36，48——患侧为左脚是负，患侧为右脚是正
    data,week,count,delta=footscan_reader.get_data('walk','vicon',name_week[0],name_week[1],augment=1)
    if type(data)!=int:
        print('下面把这次回访的同一趟的步态数据和足压数据匹配录入')
        num=0
        for i in range(len(count)):
            for gaitdata in gait_data_list:
                if info_dic.get(gaitdata.name)==name_week[0] and str(gaitdata.week*12)==name_week[1] and gaitdata.count==count[i]:
                    pressure_aver=torch.mean(data[i])
                    if pressure_aver==0:
                        print('该次回访第{}count的足压数据均为0, 为无效数据，不给记录'.format(count[i]))
                    else:
                        valid_input.append(data[i])
                        valid_week.append(int(week[i]))
                        valid_delta.append(delta[i])
                        valid_gaitdata.append(gaitdata)
                        num+=1
                    # #如果确保只有一个每一趟只有一个gaitdata文件，那么就可以break来减少时间
                    # break
        print('录入完毕，这一次回访一共有 {} 组数据是可作为有效数据的'.format(num))
        print('\n')

         

最终完成填充与拼接,最终得到的input data的维度为 torch.Size([162, 202, 1, 50, 35])
下面把这次回访的同一趟的步态数据和足压数据匹配录入
录入完毕，这一次回访一共有 162 组数据是可作为有效数据的


开始寻找  王晓军  在  12  week康复周次的足压数据,具体的task为walk,分组是vicon


In [ ]:
print(len(valid_gaitdata),len(valid_input),len(valid_week),len(valid_delta))
print(valid_gaitdata[0].data.shape)
print(valid_delta[0])
print(pd.value_counts(valid_week))

把输入处理为每一帧的数据组成的batch，把输出处理为norm

In [ ]:
#先对足压片段和膝关节力矩曲线捆绑打乱,同时把患侧为左和患侧为右的数据分开，分为两组

origin_list=[]

for i in range(len(valid_gaitdata)):
    origin_list.append((valid_input[i],valid_gaitdata[i],valid_delta[i],valid_week[i]))
random.shuffle(origin_list)

random_gaitdata_lacld=[]
random_input_lacld=[]
random_delta_lacld=[]
random_gaitdata_racld=[]
random_input_racld=[]
random_delta_racld=[]


for i in range(len(origin_list)):
    if(origin_list[i][3]<0):#代表患侧是左边
        random_input_lacld.append(origin_list[i][0])
        random_gaitdata_lacld.append(origin_list[i][1])
        random_delta_lacld.append(origin_list[i][2])
    else:
        random_input_racld.append(origin_list[i][0])
        random_gaitdata_racld.append(origin_list[i][1])
        random_delta_racld.append(origin_list[i][2])


print(len(random_input_lacld),len(random_input_racld))

#决定训练的task是lacld还是racld

this_input=random_input_lacld
this_gaitdata=random_gaitdata_lacld
this_delta=random_delta_lacld
# this_input=random_input_racld
# this_gaitdata=random_gaitdata_racld
# this_delta=random_delta_racld
whichfoot='R'


注意目前我们训练的task是：某一确定患侧的患者的某一个确定的脚侧片段与对应腿测膝关节力矩之间的关系

比如lacld的 L 脚与 L 腿膝关节力矩之间的关系 ---- 1st

racld的 L 脚与 L 腿膝关节力矩之间的关系 ---- 2nd

racld的 R 和 R --- 3rd

lacld 的 R-R --- 4th



In [ ]:
#处理label

label_list=[]
for i in range(len(this_gaitdata)):
    gaitdata=this_gaitdata[i].data
    if whichfoot=='L':
        print('此次预测的输入与输出是左脚足压-左腿膝关节力矩')
        for j in range(101):
            label_list.append(torch.norm(gaitdata[j,:]))
    else:
        print('此次预测的输入与输出是右脚足压-右腿膝关节力矩')
        for j in range(101,202):
            label_list.append(torch.norm(gaitdata[j,:]))
LABEL=torch.stack(label_list)
LABEL=LABEL.unsqueeze(-1)
#对label放大10倍，这样就避免了doubel和float的精度问题
LABEL=LABEL*10
print(LABEL.shape)
print(torch.mean(LABEL))

In [ ]:
#处理input——每一帧的通道扩充到5——this frame, last frame, dif_before,next_frame,dif_next
#同时要先把dif这个数据扩大10倍，使其与这一帧的数据同一个数量级，然后再把DATA的数字统一都都扩大100倍，使得和膝关节力矩一个数量级

input_list=[]
for i in range(len(this_input)):
    pressure_video=this_input[i]
    last_pressure=torch.zeros((50,35))
    index_list=[]
    #先判断task的脚测是L还是R
    if whichfoot=='L':
        index_list=range(101)
    else:
        index_list=range(101,202)
    
    for j in index_list:
        this_pressure=pressure_video[j,0,:,:]
        if j==100 or 201:
            next_pressure=torch.zeros((50,35))
        else:
            next_pressure=pressure_video[j+1,0,:,:]
        #考虑进入时间因素（用delta来表示）
        dif_former=(this_pressure-last_pressure)/ this_delta[i][0] *10
        dif_latter=(next_pressure-this_pressure)/this_delta[i][0] *10
        # print(torch.mean(this_pressure),torch.mean(dif_pressure))
        input_list.append(torch.stack([this_pressure,last_pressure,next_pressure,dif_former,dif_latter]))
        last_pressure=this_pressure
DATA=torch.stack(input_list)
DATA=DATA*100
print(DATA.shape)
print(torch.mean(DATA))


In [ ]:
'''
从数据中分离出测试集
'''
test_begin_index=DATA.shape[0]-DATA.shape[0]//10
#测试集取最后的5000帧（50组）
TRAIN_X=DATA[0:test_begin_index]
TRAIN_Y=LABEL[0:test_begin_index]
TEST_X=DATA[test_begin_index:-1]
TEST_Y=LABEL[test_begin_index:-1]


print('训练集输入和标签的大小分别为{},{};测试集输入和标签的大小分别为{},{}'.format(TRAIN_X.shape,TRAIN_Y.shape,TEST_X.shape,TEST_Y.shape))

In [ ]:
for i in range(TEST_X.shape[0]):
    print('第{}个数据的输入第一通道均值为{},第二通道均值为{},要预测的label均值为{}'.format(i,torch.mean(TEST_X[i]),torch.mean(TEST_X[i]-TEST_X[max(i-1,0)]),TEST_Y[i]))

## 开始模型构建

In [ ]:
#回归模型构建
'''
2023/4/22
处理足压片段来预测步态周期的力矩曲线

模型概况：
    金字塔conv:总共有四个copy——分别是三个conv和一个residual(shortcut-conv为(1,1))
    Maxpooling:
    BN:
    conv2:
    maxpooling:
    flatten:
    linear*3(12288-100-32-1)

输入输出：
    输入：该时刻的足压图像+该帧的变化量(batch,channel=5,height=50,width=35)
    输出：该时刻的膝关节力矩(batch)
'''
class Net(nn.Module):
    def __init__(self,batch_size,input_features,hidden_features_list,kernel_size_list):
        super(Net,self).__init__()

        self.batch_size = batch_size
        self.input_features = input_features
        self.hidden_features_list=hidden_features_list
        self.kernel_size_list=kernel_size_list

        self.BN1=nn.BatchNorm2d(input_features).to(DEVICE)
        self.pymarid_conv_1=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[0],padding='same').to(DEVICE)                              
        self.pymarid_conv_2=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[1],padding='same').to(DEVICE)
        self.pymarid_conv_3=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[2],padding='same').to(DEVICE)
        self.pymarid_conv_4=nn.Conv2d(input_features,hidden_features_list[0],kernel_size_list[3],padding='same').to(DEVICE)
        self.conv2=nn.Sequential(
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(hidden_features_list[0]*4),
            nn.Conv2d(hidden_features_list[0]*4,hidden_features_list[1],kernel_size_list[4],padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.Linear=nn.Sequential(
            nn.Linear(hidden_features_list[1]*12*8,100),
            nn.ELU(),
            nn.Linear(100,32),
            nn.ELU(),
            nn.Linear(32,1),
        ).to(DEVICE)

    def forward(self,x):
        if x.ndim!=4:
            raise ValueError('输入数据ndim不为4')
        else:
            batch,features,height,width= x.shape
            if batch!=self.batch_size or features != self.input_features:
                raise ValueError('输入数据batch size 或者 input features 与model不符')
            else:
                pyramid_conv_output_list=[self.pymarid_conv_1(x),self.pymarid_conv_2(x),self.pymarid_conv_3(x),self.pymarid_conv_4(x)]
                pyramid_conv_output=torch.cat(pyramid_conv_output_list,dim=1)
                conv2_output=self.conv2(pyramid_conv_output)
                y=self.Linear(conv2_output.reshape(batch,-1))
                return y

model = Net(BATCH_SIZE,INPUT_FEATURES,HIDDEN_FEATURES_LIST,KERNEL_SIZE_LIST).to(DEVICE)     
print(model)
a = torch.ones((BATCH_SIZE,INPUT_FEATURES,50,35)).to(DEVICE)
b = model(a)
print(b.shape)


开始训练


In [ ]:
start = time.time()
def timesince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


CRITERION = nn.MSELoss()
OPTIMIZER = torch.optim.Adam(model.parameters(), lr=LEARING_RATE)

In [ ]:
def MRE(pred,train_y):
    #输入时两个(batchsize,1)的二维tensor
    relative_error=abs(pred-train_y)/train_y
    return torch.mean(relative_error)


def train(train_x,train_y,test_x,test_y,batch_size,epoch_num,criterion,optimizer):
    train_loss=[]
    test_loss=[]
    train_MRE=[]
    test_MRE=[]
    train_NRMSE=[]
    test_NRMSE=[]
    full_batch = train_y.shape[0]
    batch_num=full_batch//batch_size
    train_x_list = torch.split(train_x,batch_size,dim=0)
    train_y_list = torch.split(train_y,batch_size,dim=0)
    early_stop_p = 0
    for i in range(epoch_num):
        #每一个epoch是对整个batch的训练
        loss_each_batch=[]
        MRE_each_batch=[]
        NRMSE_each_batch=[]
        for j in range(batch_num):
            train_x=train_x_list[j].to(DEVICE)
            train_y=train_y_list[j].to(DEVICE)
            output=model(train_x).to(DEVICE)
            output=output.float()
            train_y=train_y.float()
            loss = criterion(output,train_y)
            loss_each_batch.append(loss.data.cpu())
            optimizer.zero_grad()   
            loss.backward()
            optimizer.step()  

            MRE_each_batch.append(MRE(output,train_y).data.cpu())
            NRMSE_each_batch.append(np.sqrt(loss.data.cpu())/(torch.max(train_y)-torch.min(train_y)).data.cpu())

        NRMSE_this_epoch=np.average(NRMSE_each_batch)
        loss_this_epoch=np.average(loss_each_batch)
        MRE_this_epoch=np.average(MRE_each_batch)
        train_NRMSE.append(NRMSE_this_epoch)
        train_loss.append(loss_this_epoch)
        train_MRE.append(MRE_this_epoch)

        #上面是一个epoch的训练过程，下面是一个epoch的检测过程
        with torch.no_grad():
            test_MSE_each_batch=[]
            test_loss_each_batch=[]
            test_NRMSE_each_batch=[]
            test_x=test_x.to(DEVICE)
            test_y=test_y.to(DEVICE)
            test_x_list = torch.split(test_x,batch_size,dim=0)
            test_y_list = torch.split(test_y,batch_size,dim=0)
            #测试也要按照minibatch去测量
            for j in range(len(test_y_list)-1):
                test_output=model(test_x_list[j])
                test_loss_item=criterion(test_output,test_y_list[j])
                test_loss_each_batch.append(test_loss_item.data.cpu())
                test_MSE_each_batch.append(MRE(output,train_y).data.cpu())
                test_NRMSE_each_batch.append(np.sqrt(test_loss_item.data.cpu())/(torch.max(test_y)-torch.min(test_y)).data.cpu())
            test_NRMSE_this_epoch=np.average(test_NRMSE_each_batch)
            test_loss_this_epoch=np.average(test_loss_each_batch)
            test_MRE_this_epoch=np.average(test_MSE_each_batch)
            test_NRMSE.append(test_NRMSE_this_epoch)
            test_loss.append(test_loss_this_epoch)
            test_MRE.append(test_MRE_this_epoch)


        print('Epoch [{}/{}],  Train loss(MSE): {:.4f}, TEST loss(MSE): {:.4f},,time: {}'
                        .format(i, EPOCH_NUM, loss_this_epoch,test_loss_this_epoch,timesince(start)))
        
        #early_stop
        if i%3==0:
            if test_loss_this_epoch>test_loss[max(0,i-5)]:
                early_stop_p+=1
        if early_stop_p>4:
            break


    return train_loss,test_loss,train_NRMSE,test_NRMSE,train_MRE,test_MRE


train_loss,test_loss,train_NRMSE,test_NRMSE,train_MRE,test_MRE=train(TRAIN_X,TRAIN_Y,TEST_X,TEST_Y,BATCH_SIZE,EPOCH_NUM,CRITERION,OPTIMIZER)

对数据进行单独点的测试，并进行可视化

In [ ]:
'''训练集和测试集的回归结果'''
train_ref=TRAIN_Y[0:1000,0].data.cpu()
train_x=TRAIN_X[0:1000].to(DEVICE)
test_ref=TEST_Y[0:1000,0].data.cpu()
test_x=TEST_X[0:1000].to(DEVICE)

def regression_curve(x,ref,model,title):
    x_list=torch.split(x,BATCH_SIZE,dim=0)
    pred_list=[]
    for i in range(len(x_list)-1):
        pred_list.append(model(x_list[i]))
    pred=torch.cat(pred_list,dim=0)
    pred=pred.data.cpu()
    # print(pred.shape,ref.shape)
    # for i in range(100):
    #     print('第{}个数据的输入第一通道均值为{},第二通道均值为{},预测值是{},label真值为{}'.format(i,torch.mean(x[i]),torch.mean(x[i]-x[max(i-1,0)]),pred[i],ref[i]))
    plt.plot(pred[101:202],label='predicted moment')
    plt.plot(ref[101:202],label='reference moment')
    plt.xlabel('time')
    plt.ylabel('knee moment')
    plt.title(title)
    plt.legend()
    plt.show()

def coefficient_deter(x,ref,model):
    x_list=torch.split(x,BATCH_SIZE,dim=0)
    pred_list=[]
    for i in range(len(x_list)-1):
        pred_list.append(model(x_list[i]))
    pred=torch.cat(pred_list,dim=0)
    pred=pred.data.cpu()
    pred1=pred[101:202].float()
    pred1=torch.squeeze(pred1,1)
    y1=ref[101:202].float()
    error=(pred1-y1)*(pred1-y1)
    aver=torch.sum(y1)/101
    dis=(y1-aver)*(y1-aver)
    R2=1-torch.sum(error)/torch.sum(dis)
    return R2


regression_curve(train_x,train_ref,model,'training set')
regression_curve(test_x,test_ref,model,'test set')
print('coefficient of determination: example from training set {} ; example from test set{}'.format(coefficient_deter(train_x,train_ref,model),coefficient_deter(test_x,test_ref,model)))

In [ ]:
plt.plot(test_loss,label='test loss')
plt.plot(train_loss,label='train loss')
plt.xlabel('epoch_num')
plt.ylabel('MSE(absolute value)')
plt.title('MSE_loss')
plt.legend()
plt.show()

# plt.plot(test_NRMSE,label='test_nrmse')
# plt.plot(train_NRMSE,label='train_nrmse')
# plt.xlabel('epoch_num')
# plt.ylabel('Noramlized root mean square error')
# plt.title('per cent')
# plt.legend()
# plt.show()

print('最后模型的NRMSE在训练集上是{},测试集上是{}'.format(train_NRMSE[-1],test_NRMSE[-1]))
print('最后模型的MRE在训练集上是{},测试集上是{}'.format(train_MRE[-1],test_MRE[-1]))
